In the data folder there is a data file containing the daily weather summary for new york. 

read it in
count the number of inspections per day
join the result with the daily weather (precipitation (PRCP and SNOW) and temperature (TMAX, TMIN, and TAVG). 

then calculate the average number of daily inspections for days when the precipitation is nonzero, and the average number of daily inspaections for days when the precipitation is zero.

In [ ]:
import datetime
import pandas as pd
import os
import numpy as np

In [ ]:
os.chdir('../../')
print(os.listdir('.'))

In [ ]:
os.chdir('exercises/05_joining_data/data') 
print(os.listdir('.'))

In [ ]:
csv_name = 'NY_rodent_inspections_sample.csv'
rodent_dataframe = pd.read_csv(csv_name)
rodent_dataframe.head()

In [ ]:
weather_csv_name = 'ny_weather_data_daily.csv'
weather_dataframe = pd.read_csv(weather_csv_name)
weather_csv_name.head()